In [41]:
import os
import io
import PySimpleGUI as sg
from PIL import Image

font = ("Roboto", 20)
sg.theme('Dark Blue')

cwd = os.path.expanduser('~/Pictures')
file_types = [("JPEG (*.jpg)", "*.jpg"),
              ("All files (*.*)", "*.*")]

allign_center = [  
    [
        sg.Text("Image Sample"),
    ],
    [
        sg.Image(key="-IMAGE_EXAMPLE_1-"),
        sg.Image(key="-IMAGE_EXAMPLE_3-"),
        sg.Image(key="-IMAGE_EXAMPLE_2-")
    ],
    [
        sg.Text("Confident = "),
    ],
    [
        sg.Image(key="-IMAGE_PREDICTION-")
    ],
    [
        sg.Text("Image File Path"),
        sg.Input(size=(25, 1), enable_events=True, key="-FILE_PATH-"),
        sg.FileBrowse(initial_folder=cwd, file_types=file_types),
    ],
    [
        sg.Button("Submit Image", font=("Roboto", 14))
    ],
]

main_menu_window = [
    [sg.Push(), sg.Column(allign_center, element_justification='c')]
]

window = sg.Window("Dog Breed Classification", main_menu_window)
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, "Exit"):
        break
    elif event == 'Submit Image':
        filename = values["-FILE_PATH-"]
        if os.path.exists(filename):
            image = Image.open(values["-FILE_PATH-"])
            image.thumbnail((400, 400))
            bio = io.BytesIO()
            image.save(bio, format="PNG")
            window["-IMAGE_PREDICTION-"].update(data=bio.getvalue())
            window["-IMAGE_EXAMPLE_1-"].update(data=bio.getvalue())
            window["-IMAGE_EXAMPLE_2-"].update(data=bio.getvalue())
            window["-IMAGE_EXAMPLE_3-"].update(data=bio.getvalue())

window.close()

In [42]:
import skimage
import imageio
import keras
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from imutils import paths
from skimage.io import imread
from skimage import io
from skimage.color import rgb2gray
# from google.colab.patches import cv2_imshow

import os
from os import listdir
from os.path import isfile, join
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


features = []
labels = []

def extractFeature(img):
  image = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  # plt.imshow(image, cmap='gray')
  co_matrix = skimage.feature.graycomatrix(image, [5], [0], levels=256, symmetric=True, normed=True)

  # texture_features = np.array([skimage.feature.graycoprops(co_matrix, prop) for prop in ['contrast', 'correlation', 'energy', 'homogeneity']])


  contrast = skimage.feature.graycoprops(co_matrix, 'contrast')
  correlation = skimage.feature.graycoprops(co_matrix, 'correlation')
  energy = skimage.feature.graycoprops(co_matrix, 'energy')
  homogeneity = skimage.feature.graycoprops(co_matrix, 'homogeneity')

  texture_features = np.array((contrast, correlation, energy, homogeneity))

  features.append(texture_features)

  # Print the texture features
  # print("Contrast:", contrast)
  # print("Correlation:", correlation)
  # print("Energy:", energy)
  # print("Homogeneity:", homogeneity)

  return contrast, correlation, energy, homogeneity


folder_list = ['insang', 'kawung', 'megamendung', 'parang', 'poleng']

for i in range(len(folder_list)):
  motif = folder_list[i]
  folder_path = "/content/drive/MyDrive/dataset_batik/" + motif
  img_list = list(paths.list_images(folder_path))
  sum_contrast = 0
  sum_correlation = 0
  sum_energy = 0
  sum_homogeneity = 0
  for j in range(len(img_list)):
    labels.append(motif)
    img_path = img_list[j]
    img = cv.imread(img_path)
    # print(img_path)
    contrast, correlation, energy, homogeneity = extractFeature(img)
    sum_contrast += contrast
    sum_correlation += correlation
    sum_energy += energy
    sum_homogeneity += homogeneity

  avg_contrast = sum_contrast / len(img_list)
  avg_correlation = sum_correlation / len(img_list)
  avg_energy = sum_energy / len(img_list)
  avg_homogeneity = sum_homogeneity / len(img_list)

  print(folder_path)
  print("avg_contrast = ", avg_contrast)
  print("avg_correlation = ", avg_correlation)
  print("avg_energy = ", avg_energy)
  print("avg_homogeneity = ", avg_homogeneity)
  print()


features = np.array(features)
labels = np.array(labels)
# print(features)
# print(labels)

features = features.reshape(features.shape[0], -1)

# Pisahkan data menjadi data pelatihan dan pengujian
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Inisialisasi model KNN
knn_model = KNeighborsClassifier(n_neighbors=7)

# Latih model dengan data pelatihan
knn_model.fit(x_train, y_train)

# Prediksi label untuk data pengujian
knn_predictions = knn_model.predict(x_test)
print(knn_predictions)

# Evaluasi akurasi model
print(y_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print(f"KNN Accuracy: {knn_accuracy}")

ModuleNotFoundError: No module named 'keras'